# Imports

In [1]:
import numpy as np
import itertools
import time
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
# import pixiedust
from numba import jit, njit, cuda, vectorize, float64, float32, guvectorize
import numba
import os
import cProfile as profile
# import cupy as cp

# Introduction

Consider the following equation:
$$\Delta_p \Psi(x,y)=f(x,y)$$

$x \in [0,1],\ y \in [0,1]$ with *Dirichlet* BC: $\Psi(0,y) = 0$, $\Psi(1,y) = 0$, $\Psi(x,0) = 0$ and $\Psi(x,1) = 0$.

For this first attempt, we will take $p=2$.

# Defining functions

In [2]:
HN_CONST=10

## Arith

In [3]:
@cuda.jit(device=True)
def g_mul_num_arr(x, arr, result):
    for i in range(arr.shape[0]):
        result[i] = x * arr[i]
    return result

@cuda.jit(device=True)
def g_mul_2(arr1, arr2, result):
    for i in range(arr1.shape[0]):
        result[i] = arr1[i] * arr2[i]
    return result

@cuda.jit(device=True)
def g_add_2(arr1, arr2, result):
    for i in range(arr1.shape[0]):
        result[i] = arr1[i] + arr2[i]
    return result

@cuda.jit(device=True)
def g_add_3(arr1, arr2, arr3, result):
    for i in range(arr1.shape[0]):
        result[i] = arr1[i] + arr2[i] + arr3[i]
    return result

#returns x-arr
@cuda.jit(device=True)
def g_sub_num_arr(x, arr, result):
    for i in range(arr.shape[0]):
        result[i] = x - arr[i]
    return result

#returns arr1-arr2
@cuda.jit(device=True)
def g_sub_2(arr1, arr2, result):
    for i in range(arr1.shape[0]):
        result[i] = arr1[i] - arr2[i]
    return result

@cuda.jit(device=True)
def g_sum(arr):
    s = 0
    for i in range(arr.shape[0]):
        s += arr[i]
    return s

@cuda.jit(device=True)
def g_pow(arr, n, result):
    for i in range(arr.shape[0]):
        result[i] = arr[i]**n
    return result


## Sigmoid

Sigmoid $\sigma(x) = \frac{1}{1+e^{-x}}$ and its derrivatives.

Sigmoid with parameter $t$
$$ \sigma_t(x) = \frac{1}{1+e^{-tx}}$$

In [4]:
@cuda.jit(device=True)
def sig(x, result):
    for i in range(x.shape[0]):
        result[i] = 1 / (1 + np.e**(-x[i]))
    return result

@njit
def sig_orig(x):
    return 1 / (1 + np.exp(-x))

@cuda.jit(device=True)
def sig1(x, result):
    sig_x = cuda.local.array(shape=HN_CONST, dtype=float64)
    sig_minus = cuda.local.array(shape=HN_CONST, dtype=float64)
    
    sig_x = sig(x, sig_x)
    sig_minus = g_sub_num_arr(1, sig_x, sig_minus)
    
    for i in range(HN_CONST):
        result[i] = sig_x[i] * sig_minus[i]
    return result

@njit
def sig1_orig(x):
    return sig_orig(x) * (1 - sig_orig(x))

@cuda.jit(device=True)
def sig2(x, result):
    sig_x = cuda.local.array(shape=HN_CONST, dtype=float64)
    sig_1_x = cuda.local.array(shape=HN_CONST, dtype=float64)
    
    sig_x = sig(x, sig_x)
    sig_1_x = sig1(x, sig_1_x)
    
    for i in range(x.shape[0]):
        result[i] = sig_1_x[i] - 2*sig_x[i]*sig_1_x[i]
    
    return result

@njit
def sig2_orig(x):
    return (sig1_orig(x) - 2*sig_orig(x)*sig1_orig(x))

@cuda.jit(device=True)
def sig3(x, result):
    sig_x = cuda.local.array(shape=HN_CONST, dtype=float64)
    sig_1_x = cuda.local.array(shape=HN_CONST, dtype=float64)
    sig_2_x = cuda.local.array(shape=HN_CONST, dtype=float64)

    sig_x = sig(x, sig_x)
    sig_1_x = sig1(x, sig_1_x)
    sig_2_x = sig2(x, sig_2_x)
    
    for i in range(x.shape[0]):
        result[i] = sig_2_x[i] - 2 * (sig_1_x[i]**2 + sig_x[i] * sig_2_x[i])
    
    return result

@njit
def sig3_orig(x):
    return (sig2_orig(x) - 2 * (sig1_orig(x)**2 + sig_orig(x) * sig2_orig(x)))

@cuda.jit(device=True)
def sig_pr(x, k, result):
    if k==0:
        result = sig(x, result)
    if k==1:
        result = sig1(x, result)
    if k==2:
        result = sig2(x, result)
    if k==3:
        result = sig3(x, result)
#     return result

@njit
def sig_pr_orig(x, k):
    if k==0:
        return sig_orig(x)
    if k==1:
        return sig1_orig(x)
    if k==2:
        return sig2_orig(x)
    if k==3:
        return sig3_orig(x)

## RHS

The right side of the equation:  $\Delta\Psi(x,y) = 6 x (x-1) (1-2 y) + 2 y (y-1) (1-2 y)$

In [5]:
K=10
@njit
def f(x, y):
    return K * (6*(x-1)*x*(1-2*y) + 2*(y-1)*y*(1-2*y))

The analytic solution is given by: $\ \Psi_a(x,y) = x (1-x) y (1-y) (1-2 y)$

In [6]:
@njit
def psi_a(x, y):
    return K * x*(1-x)*y*(1-y)*(1-2*y)

$$\frac{\partial}{\partial x} \Psi_a = (1-2x) y (1-y) (1-2 y)$$

In [7]:
@njit
def psi_a_dx(x,y):
    return K * (1-2*x)*y*(1-y)*(1-2*y)

$$\frac{\partial}{\partial y} \Psi_a = x (1-x) (1-6y + 6y^2) $$

In [8]:
@njit
def psi_a_dy(x,y):
    return K * x*(1-x)*(1-6*y+6*(y**2))

## Neural Network

The output of neural network $N(x,y,\vec{p})$, where $\vec{p} = [w, u, v]$:
$$N = \sum_i^H v_i \sigma(z_i) \text{, where } z_i = w_{i0} x + w_{i1} y + u_i$$

In [9]:
@cuda.jit(device=True)
def z(x, y, p00, p01, p1, result):
    z_x = cuda.local.array(shape=HN_CONST, dtype=float64)
    z_y = cuda.local.array(shape=HN_CONST, dtype=float64)

    z_x = g_mul_num_arr(x, p00, z_x)
    z_y = g_mul_num_arr(y, p01, z_y)
    g_add_3(z_x, z_y, p1, result)

    return result

@njit
def z_orig(x, y, p00, p01, p1):
    z_x = x * p00
    z_y = y * p01
    z_ = z_x + z_y + p1

    return z_


@cuda.jit(device=True)
def N(x, y, p00, p01, p1, p2):
    n_res = cuda.local.array(shape=HN_CONST, dtype=float64)
    n_res = z(x, y, p00, p01, p1, n_res)
    n_res = sig(n_res, n_res)
    n_res = g_mul_2(n_res, p2, n_res)
    return g_sum(n_res)

@njit
def N_orig(x, y, p00, p01, p1, p2):
    return np.sum(sig_orig(z_orig(x, y, p00, p01, p1)) * p2)

$$\frac{\partial^k N}{\partial x_j^k} = \sum_{i=1}^H v_i w_{ij}^k \sigma^{(k)}$$

In [10]:
@cuda.jit(device=True)
def dN_dxj_k(x, y, p00, p01, p1, p2, j, k):
    wj = p00 if j==0 else p01
    z_ = cuda.local.array(shape=HN_CONST, dtype=float64)
    z_ = z(x, y, p00, p01, p1, z_)
    sig_pr_ = cuda.local.array(shape=HN_CONST, dtype=float64)
    sig_pr(z_, k, sig_pr_)
    
    s = 0
    for i in range(HN_CONST):
        s += p2[i] * (wj[i]**k) * sig_pr_[i]
    
    return s

@njit
def dN_dxj_k_orig(x, y, p00, p01, p1, p2, j, k):
    wj = p00 if j==0 else p01
    z_ = z_orig(x, y, p00, p01, p1)
    
    return np.sum(p2 * (wj**k) * sig_pr_orig(z_, k))

$$\frac{\partial N}{\partial w_j} = x_j v \sigma '$$

In [11]:
@cuda.jit(device=True)
def dN_dwj(x, y, p00, p01, p1, p2, j, result):
    xj = x if j==0 else y
    sig1_z = cuda.local.array(shape=HN_CONST, dtype=float64)
    sig1_z = z(x, y, p00, p01, p1, sig1_z)
    sig1_z = sig1(sig1_z, sig1_z)
    for i in range(HN_CONST):
        result[i] = xj * p2[i] * sig1_z[i]
    return result

@njit
def dN_dwj_orig(x, y, p00, p01, p1, p2, j):
    xj = x if j==0 else y
    z_ = z_orig(x, y, p00, p01, p1)
    return xj * p2 * sig1_orig(z_)

$$ \frac{\partial}{\partial w_j} \frac{\partial N}{\partial x_k} = x_j v w_k \sigma'' + v_i \sigma' \quad\text{ if } j = k$$

$$ \frac{\partial}{\partial w_j} \frac{\partial N}{\partial x_k} = x_j v w_k \sigma'' \quad\text{ if } j \neq k$$

In [12]:
@cuda.jit(device=True)
def d_dwj_dN_dxk(x, y, p00, p01, p1, p2, j, k):
    xj = x if j==0 else y
    wk = p00 if k==0 else p01
    jk = 1 if j==k else 0
    z_ = z(x, y, p00, p01, p1)
    return xj * p2 * wk * sig2(z_) + jk * p2 * sig1(z_)

$$ \frac{\partial}{\partial w_j} \frac{\partial^2 N}{\partial x_k^2} = x_j v w_k^2 \sigma^{(3)} + 2 v w_k \sigma'' \quad\text{ if } j = k $$

$$ \frac{\partial}{\partial w_j} \frac{\partial^2 N}{\partial x_k^2} = x_j v w_k^2 \sigma^{(3)} \quad\text{ if } j \neq k $$

In [13]:
@cuda.jit(device=True)
def d_dwj_dN2_dxk2(x, y, p00, p01, p1, p2, j, k, result):
    xj = x if j==0 else y
    wk = p00 if k==0 else p01
    jk = 1 if j==k else 0
    
    z_ = cuda.local.array(shape=HN_CONST, dtype=float64)
    z_ = z(x, y, p00, p01, p1, z_)
    sig_2 = cuda.local.array(shape=HN_CONST, dtype=float64)
    sig_2 = sig2(z_, sig_2)
    sig_3 = cuda.local.array(shape=HN_CONST, dtype=float64)
    sig_3 = sig3(z_, sig_3)
    
    for i in range(HN_CONST):
        result[i] = xj * p2[i] * (wk[i]**2) * sig_3[i] + \
                    jk * 2 * p2[i] * wk[i] * sig_2[i]
    
    return result

@njit
def d_dwj_dN2_dxk2_orig(x, y, p00, p01, p1, p2, j, k):
    xj = x if j==0 else y
    wk = p00 if k==0 else p01
    jk = 1 if j==k else 0
    z_ = z_orig(x, y, p00, p01, p1)
    return xj * p2 * (wk**2) * sig3_orig(z_) + jk * 2 * p2 * wk * sig2_orig(z_)

$$ \frac{\partial}{\partial u} \frac{\partial^k}{\partial x_j^k} N = v w_j^k \sigma^{(k+1)} $$

In [14]:
@cuda.jit(device=True)
def d_du_dkN(x, y, p00, p01, p1, p2, j, k, result):
    wj = p00 if j==0 else p01
    z_ = cuda.local.array(shape=HN_CONST, dtype=float64)
    z_ = z(x, y, p00, p01, p1, z_)
    
    sig_pr_ = cuda.local.array(shape=HN_CONST, dtype=float64)
    sig_pr(z_, k+1, sig_pr_)
    
    for i in range(HN_CONST):
        result[i] = p2[i] * (wj[i]**k) * sig_pr_[i]

    return result

@njit
def d_du_dkN_orig(x, y, p00, p01, p1, p2, j, k):
    wj = p00 if j==0 else p01
    z_ = z_orig(x, y, p00, p01, p1)
    return p2 * (wj**k) * sig_pr_orig(z_, k+1)

$$ \frac{\partial}{\partial v} \frac{\partial^k}{\partial x_j^k} N = w_j^k \sigma^{(k)} $$

In [15]:
@cuda.jit(device=True)
def d_dv_dkN(x, y, p00, p01, p1, p2, j, k, result):
    wj = p00 if j==0 else p01
    z_ = cuda.local.array(shape=HN_CONST, dtype=float64)
    z_ = z(x, y, p00, p01, p1, z_)
    
    sig_pr_ = cuda.local.array(shape=HN_CONST, dtype=float64)
    sig_pr(z_, k, sig_pr_)
    
    for i in range(HN_CONST):
        result[i] = (wj[i]**k) * sig_pr_[i]

    return result

@njit
def d_dv_dkN_orig(x, y, p00, p01, p1, p2, j, k):
    wj = p00 if j==0 else p01
    z_ = z_orig(x, y, p00, p01, p1)
    return (wj**k) * sig_pr_orig(z_, k)

## Cost function

$$E[\vec{p}] = \sum_{i \in \hat{D}} \left\{ \frac{\partial^2 N}{\partial x^2} + \frac{\partial^2 N}{\partial y^2} - f(x,y) \right\}^2 
           +  \sum_{i \in \partial \hat{D}} N^2$$

In [16]:
BC=1

In [17]:
@cuda.jit(device=True)
def error_term1(x, y, p00, p01, p1, p2):
    ans = dN_dxj_k(x, y, p00, p01, p1, p2, 0, 2) + \
            dN_dxj_k(x, y, p00, p01, p1, p2, 1, 2)  -  f(x, y)
    return ans

@njit
def error_term1_orig(x, y, p00, p01, p1, p2):
    ans = dN_dxj_k_orig(x, y, p00, p01, p1, p2, 0, 2) + \
            dN_dxj_k_orig(x, y, p00, p01, p1, p2, 1, 2)  -  f(x, y)
    return ans

In [44]:
@njit
def cost(points, boundary_points, p00, p01, p1, p2):
    et1 = np.zeros(points.shape[0])
    et2 = np.zeros(boundary_points.shape[0])
    
    for i, pnt in enumerate(points):
        et1[i] = error_term1_orig(pnt[0], pnt[1], p00, p01, p1, p2)**2
    
    for i, pnt in enumerate(boundary_points):
        et2[i] = N_orig(pnt[0], pnt[1], p00, p01, p1, p2)**2
    
    cost = np.sum(et1) + BC*np.sum(et2)
    
    return cost

In [19]:
# @cuda.jit
# def cost_et1(points, p00, p01, p1, p2, result):
#     pos = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
#     if pos >= points.shape[0]:
#         return
    
#     result[pos] = error_term1(points[pos][0], points[pos][1], p00, p01, p1, p2)**2

# @cuda.jit
# def cost_et2(points, p00, p01, p1, p2, result):
#     pos = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
#     if pos >= points.shape[0]:
#         return
    
#     result[pos] = N(points[pos][0], points[pos][1], p00, p01, p1, p2)**2
    
# def cost(points, boundary_points, p00, p01, p1, p2):
#     et1 = np.zeros(points.shape[0])
#     et2 = np.zeros(boundary_points.shape[0])
#     threadsperblock = 64
    
#     blockspergrid_et1 = (points.shape[0] + (threadsperblock - 1)) // threadsperblock
#     cost_et1[blockspergrid_et1, threadsperblock](points, p00, p01, p1, p2, et1)
    
#     blockspergrid_et2 = (points.shape[0] + (threadsperblock - 1)) // threadsperblock
#     cost_et2[blockspergrid_et1, threadsperblock](boundary_points, p00, p01, p1, p2, et2)
    
#     cost = np.sum(et1) + BC*np.sum(et2)
    
#     return cost

In [20]:
def relative_err_without_points(p00, p01, p1, p2, nx=100):
    all_points = np.array(list(itertools.product(np.linspace(0, 1, nx), np.linspace(0, 1, nx))))
    return relative_err(p00, p01, p1, p2, all_points)

@njit
def relative_err(p00, p01, p1, p2, all_points):
    dOmega = 1. / len(all_points)
    
    tr1 = np.zeros(all_points.shape[0])
    tr2 = np.zeros(all_points.shape[0])
    ana1 = np.zeros(all_points.shape[0])
    ana2 = np.zeros(all_points.shape[0])
    
    for i, pnt in enumerate(all_points):
        tr1[i] = dOmega * (np.abs(N_orig(pnt[0], pnt[1], p00, p01, p1, p2) - psi_a(pnt[0],pnt[1]))**2)

    for i, pnt in enumerate(all_points):
        tr2[i] = dOmega * ((dN_dxj_k_orig(pnt[0],pnt[1],p00,p01,p1,p2,0,1)-psi_a_dx(pnt[0],pnt[1]))**2 +
                           (dN_dxj_k_orig(pnt[0],pnt[1],p00,p01,p1,p2,1,1)-psi_a_dy(pnt[0],pnt[1]))**2)

    for i, pnt in enumerate(all_points):
        ana1[i] = dOmega * (np.abs(psi_a(pnt[0],pnt[1]))**2)
        
    for i, pnt in enumerate(all_points):
        ana2[i] = dOmega* (psi_a_dx(pnt[0],pnt[1])**2 + psi_a_dy(pnt[0],pnt[1])**2)
    
    rel_err = (np.sum(tr1) + np.sum(tr2))**(1/2) / (np.sum(ana1) + np.sum(ana2))**(1/2)

    return rel_err

# Gradients

In [21]:
@njit
def grad_ax0_sum(t1,t2):
    return np.sum(t1, axis=0) + BC*np.sum(t2, axis=0)

$$ \frac{\partial E[\vec{p}]}{\partial w_j} = \sum_{i \in \hat{D}} \left\{ 2 \text{ (error_term1) } \left( \frac{\partial}{\partial w_j} \frac{\partial^2 N}{\partial x^2} + \frac{\partial}{\partial w_j} \frac{\partial^2 N}{\partial y^2} \right) \right\}  +  \sum_{i \in \partial \hat{D}} 2 N \frac{\partial N}{\partial w_j}$$

In [22]:
@njit
def dE_dwj_orig(points, boundary_points, p00, p01, p1, p2, j):
    t1 = np.zeros((points.shape[0], p00.shape[0]))
    t2 = np.zeros((boundary_points.shape[0], p00.shape[0]))
    
    for i, pnt in enumerate(points):
        t1[i] = 2 * error_term1_orig(pnt[0],pnt[1],p00, p01, p1, p2) * (
                d_dwj_dN2_dxk2_orig(pnt[0],pnt[1],p00, p01, p1, p2,j,0) + 
                d_dwj_dN2_dxk2_orig(pnt[0],pnt[1],p00, p01, p1, p2,j,1))
        
    for i, pnt in enumerate(boundary_points):
        t2[i] = 2 * N_orig(pnt[0],pnt[1],p00, p01, p1, p2) * dN_dwj_orig(pnt[0],pnt[1],p00, p01, p1, p2,j)
    
    grad = np.sum(t1, axis=0) + BC*np.sum(t2, axis=0)
    
    return grad

In [23]:
@cuda.jit
def dE_dwj_t1(points, p00, p01, p1, p2, j, result):
    pos = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    
    if pos >= points.shape[0]:
        return
    dnx = cuda.local.array(shape=HN_CONST, dtype=float64)
    dnx = d_dwj_dN2_dxk2(points[pos][0], points[pos][1], p00, p01, p1, p2, j, 0, dnx)
    dny = cuda.local.array(shape=HN_CONST, dtype=float64)
    dny = d_dwj_dN2_dxk2(points[pos][0], points[pos][1], p00, p01, p1, p2, j, 1, dny)
    et1 = error_term1(points[pos][0], points[pos][1], p00, p01, p1, p2)

    t1 = g_add_2(dnx, dny, dnx)
    t1 = g_mul_num_arr(2*et1, dnx, t1)
    for i in range(HN_CONST):
        result[pos][i] = t1[i]

In [24]:
@cuda.jit
def dE_dwj_t2(points, p00, p01, p1, p2, j, result):
    pos = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    
    if pos >= points.shape[0]:
        return
    n_ = N(points[pos][0], points[pos][1], p00, p01, p1, p2)
    dn_dwj_ = cuda.local.array(shape=HN_CONST, dtype=float64)
    dn_dwj_ = dN_dwj(points[pos][0], points[pos][1], p00, p01, p1, p2, j, dn_dwj_)
    for i in range(HN_CONST):
        result[pos][i] = 2 * n_ * dn_dwj_[i]

In [25]:
def dE_dwj(points, boundary_points, p00, p01, p1, p2, j):
    t1 = np.zeros((points.shape[0], p00.shape[0]))
    t2 = np.zeros((boundary_points.shape[0], p00.shape[0]))
    threadsperblock = 64
    
    blockspergrid_t1 = (points.shape[0] + (threadsperblock - 1)) // threadsperblock
    dE_dwj_t1[blockspergrid_t1, threadsperblock](points, p00, p01, p1, p2, j, t1)
    
    blockspergrid_t2 = (points.shape[0] + (threadsperblock - 1)) // threadsperblock
    dE_dwj_t2[blockspergrid_t2, threadsperblock](boundary_points, p00, p01, p1, p2, j, t2)
    
    grad = grad_ax0_sum(t1,t2)
    
    return grad

$$ \frac{\partial E[\vec{p}]}{\partial u} = \sum_{i \in \hat{D}} \left\{ 2 \text{ (error_term1) } \left( \frac{\partial}{\partial u} \frac{\partial^2 N}{\partial x^2} + \frac{\partial}{\partial u} \frac{\partial^2 N}{\partial y^2} \right) \right\} +  \sum_{i \in \partial \hat{D}} 2 N \frac{\partial N}{\partial u}$$

In [26]:
@njit
def dE_du_orig(points, boundary_points, p00, p01, p1, p2):
    t1 = np.zeros((points.shape[0], p1.shape[0]))
    t2 = np.zeros((boundary_points.shape[0], p1.shape[0]))
    
    for i, pnt in enumerate(points):
        t1[i] = 2 * error_term1_orig(pnt[0],pnt[1],p00, p01, p1, p2) * \
                (d_du_dkN_orig(pnt[0],pnt[1],p00, p01, p1, p2,0,2) + \
                 d_du_dkN_orig(pnt[0],pnt[1],p00, p01, p1, p2,1,2))
        
    for i, pnt in enumerate(boundary_points):
        t2[i] = 2 * N_orig(pnt[0],pnt[1],p00, p01, p1, p2) * d_du_dkN_orig(pnt[0],pnt[1],p00, p01, p1, p2,0,0)
    
    grad = np.sum(t1, axis=0) + BC*np.sum(t2, axis=0)
    
    return grad

In [27]:
@cuda.jit
def dE_du_t1(points, p00, p01, p1, p2, result):
    pos = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    
    if pos >= points.shape[0]:
        return
    dnx = cuda.local.array(shape=HN_CONST, dtype=float64)
    dnx = d_du_dkN(points[pos][0], points[pos][1], p00, p01, p1, p2, 0, 2, dnx)
    dny = cuda.local.array(shape=HN_CONST, dtype=float64)
    dny = d_du_dkN(points[pos][0], points[pos][1], p00, p01, p1, p2, 1, 2, dny)
    et1 = error_term1(points[pos][0], points[pos][1], p00, p01, p1, p2)

    t1 = g_add_2(dnx, dny, dnx)
    t1 = g_mul_num_arr(2*et1, dnx, t1)
    for i in range(HN_CONST):
        result[pos][i] = t1[i]

In [28]:
@cuda.jit
def dE_du_t2(points, p00, p01, p1, p2, result):
    pos = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    
    if pos >= points.shape[0]:
        return
    
    n_ = N(points[pos][0], points[pos][1], p00, p01, p1, p2)
    dn_du_ = cuda.local.array(shape=HN_CONST, dtype=float64)
    dn_du_ = d_du_dkN(points[pos][0], points[pos][1], p00, p01, p1, p2, 0, 0, dn_du_)
    for i in range(HN_CONST):
        result[pos][i] = 2 * n_ * dn_du_[i]

In [29]:
def dE_du(points, boundary_points, p00, p01, p1, p2):
    t1 = np.zeros((points.shape[0], p00.shape[0]))
    t2 = np.zeros((boundary_points.shape[0], p00.shape[0]))
    threadsperblock = 64
    
    blockspergrid_t1 = (points.shape[0] + (threadsperblock - 1)) // threadsperblock
    dE_du_t1[blockspergrid_t1, threadsperblock](points, p00, p01, p1, p2, t1)
    
    blockspergrid_t2 = (points.shape[0] + (threadsperblock - 1)) // threadsperblock
    dE_du_t2[blockspergrid_t2, threadsperblock](boundary_points, p00, p01, p1, p2, t2)
    
    grad = grad_ax0_sum(t1,t2)
    
    return grad

$$ \frac{\partial E[\vec{p}]}{\partial v} = \sum_{i \in \hat{D}} \left\{ 2 \text{ (error_term1) } \left( \frac{\partial}{\partial v} \frac{\partial^2 N}{\partial x^2} + \frac{\partial}{\partial v} \frac{\partial^2 N}{\partial y^2} \right) \right\}  +  \sum_{i \in \partial \hat{D}} 2 N \frac{\partial N}{\partial v}$$

In [30]:
@njit
def dE_dv_orig(points, boundary_points, p00, p01, p1, p2):
    t1 = np.zeros((points.shape[0], p2.shape[0]))
    t2 = np.zeros((boundary_points.shape[0], p2.shape[0]))
    
    for i, pnt in enumerate(points):
        t1[i] = 2 * error_term1_orig(pnt[0],pnt[1],p00, p01, p1, p2) * \
                (d_dv_dkN_orig(pnt[0],pnt[1],p00, p01, p1, p2,0,2) + \
                 d_dv_dkN_orig(pnt[0],pnt[1],p00, p01, p1, p2,1,2))
        
    for i, pnt in enumerate(boundary_points):
        t2[i] = 2 * N_orig(pnt[0],pnt[1],p00, p01, p1, p2) * d_dv_dkN_orig(pnt[0],pnt[1],p00, p01, p1, p2,0,0)
    
    grad = np.sum(t1, axis=0) + BC*np.sum(t2, axis=0)
    
    return grad

In [31]:
@cuda.jit
def dE_dv_t1(points, p00, p01, p1, p2, result):
    pos = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    
    if pos >= points.shape[0]:
        return
    dnx = cuda.local.array(shape=HN_CONST, dtype=float64)
    dnx = d_dv_dkN(points[pos][0], points[pos][1], p00, p01, p1, p2, 0, 2, dnx)
    dny = cuda.local.array(shape=HN_CONST, dtype=float64)
    dny = d_dv_dkN(points[pos][0], points[pos][1], p00, p01, p1, p2, 1, 2, dny)
    et1 = error_term1(points[pos][0], points[pos][1], p00, p01, p1, p2)

    t1 = g_add_2(dnx, dny, dnx)
    t1 = g_mul_num_arr(2*et1, dnx, t1)
    for i in range(HN_CONST):
        result[pos][i] = t1[i]

In [32]:
@cuda.jit
def dE_dv_t2(points, p00, p01, p1, p2, result):
    pos = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    
    if pos >= points.shape[0]:
        return
    
    n_ = N(points[pos][0], points[pos][1], p00, p01, p1, p2)
    dn_dv_ = cuda.local.array(shape=HN_CONST, dtype=float64)
    dn_dv_ = d_dv_dkN(points[pos][0], points[pos][1], p00, p01, p1, p2, 0, 0, dn_dv_)
    for i in range(HN_CONST):
        result[pos][i] = 2 * n_ * dn_dv_[i]

In [33]:
def dE_dv(points, boundary_points, p00, p01, p1, p2):
    t1 = np.zeros((points.shape[0], p00.shape[0]))
    t2 = np.zeros((boundary_points.shape[0], p00.shape[0]))
    threadsperblock = 64
    
    blockspergrid_t1 = (points.shape[0] + (threadsperblock - 1)) // threadsperblock
    dE_dv_t1[blockspergrid_t1, threadsperblock](points, p00, p01, p1, p2, t1)
    
    blockspergrid_t2 = (points.shape[0] + (threadsperblock - 1)) // threadsperblock
    dE_dv_t2[blockspergrid_t2, threadsperblock](boundary_points, p00, p01, p1, p2, t2)
    
    grad = grad_ax0_sum(t1,t2)
    
    return grad

# NN class

In [34]:
# @njit
def get_mini_batches(points, boundary_points, batch_size):
    np.random.shuffle(points)
    np.random.shuffle(boundary_points)
    
    no_of_splits = np.ceil( (len(points) + len(boundary_points)) / batch_size)

    mini_batch_points = np.array_split(points, no_of_splits)
    mini_batch_boundary_points = np.array_split(boundary_points, no_of_splits)
    
    return mini_batch_points, mini_batch_boundary_points

In [35]:
class NNTrain:
    def __init__(self, nx=10, bx=10, hidden_nodes=10, alpha=0.01, batch_size=50,
                 beta=0.9, update_interval=50, if_rel_err=False, 
                 output_file='output/output.csv'):
        
        self.output_file = output_file
        self.training_started = False
        self.nx = nx
        self.hidden_nodes = hidden_nodes
        self.alpha = alpha
        self.batch_size = batch_size
        self.beta = beta
        self.update_interval = update_interval
        self.boundary_points = np.array(list(set(list(itertools.product([0, 1], np.linspace(0,1,bx))) +
                                        list(itertools.product(np.linspace(0,1,bx), [0, 1])))))
        pnts = list(itertools.product(np.linspace(0, 1, nx), np.linspace(0, 1, nx)))
        self.points = np.array([(x,y) for x,y in pnts if (x not in [0,1] and y not in [0,1])])
        self.cost_rate = []
        self.if_rel_err = if_rel_err
        if self.if_rel_err:
            self.rel_err = []
#         self.p = np.array([np.random.randn(2,hidden_nodes),
#                            np.random.randn(hidden_nodes),
#                            np.random.randn(hidden_nodes)])
        self.p00 = np.random.randn(hidden_nodes)
        self.p01 = np.random.randn(hidden_nodes)
        self.p1 = np.random.randn(hidden_nodes)
        self.p2 = np.random.randn(hidden_nodes)
        self.m_t = np.array([np.zeros(hidden_nodes),
                             np.zeros(hidden_nodes),
                             np.zeros(hidden_nodes),
                             np.zeros(hidden_nodes)])


    def sgd_mt(self, w, g_t, theta_0):
        #gradient descent with momentum
        self.m_t[w] = self.beta * self.m_t[w] + (1-self.beta) * g_t
        theta_0 = theta_0 - (self.alpha*self.m_t[w])
            
        return theta_0
        

    def train(self, itr=1000):
        self.training_started=True
        
        start=len(self.cost_rate)-1
        if start<1:
            start+=1
            self.cost_rate.append(cost(self.points,self.boundary_points,self.p00, self.p01, self.p1, self.p2))
            if self.if_rel_err:
                self.rel_err.append(relative_err(self.p00, self.p01, self.p1, self.p2, 
                                                 all_points=np.vstack([self.points, self.boundary_points])))

        i = start
        while i < start+itr:
            mini_batch_points, mini_batch_boundary = get_mini_batches(self.points, 
                                                                      self.boundary_points, self.batch_size)

            for mini_point, mini_boundary in zip(mini_batch_points, mini_batch_boundary):
#                 mini_point = list(mini_point)
#                 mini_boundary = list(mini_boundary)

                g_w0 = dE_dwj(mini_point, mini_boundary, self.p00, self.p01, self.p1, self.p2, 0)
                g_w1 = dE_dwj(mini_point, mini_boundary, self.p00, self.p01, self.p1, self.p2, 1)
                g_u = dE_du(mini_point, mini_boundary, self.p00, self.p01, self.p1, self.p2)
                g_v = dE_dv(mini_point, mini_boundary, self.p00, self.p01, self.p1, self.p2)

                self.pp00 = self.sgd_mt(0, g_w0, self.p00)
                self.p01 = self.sgd_mt(1, g_w1, self.p01)
                self.p1 = self.sgd_mt(2, g_u, self.p1)
                self.p2 = self.sgd_mt(3, g_v, self.p2)

            self.cost_rate.append(cost(self.points,self.boundary_points,self.p00, self.p01, self.p1, self.p2))
            cost_diff = self.cost_rate[i]-self.cost_rate[i+1]
            if self.if_rel_err:
                self.rel_err.append(relative_err(self.p00, self.p01, self.p1, self.p2, 
                                                 all_points=np.vstack([self.points, self.boundary_points])))
                rel_diff = self.rel_err[i]-self.rel_err[i+1]

            i+=1
                
                
    def save_result(self, output_name=''):
        timestr = time.strftime("%Y%m%d-%H%M")
        np.savez('output/'+ timestr + '_' + output_name +'_nn_params.npz', self.p)
        np.savez('output/'+ timestr + '_' + output_name +'_cost_rate.npz', self.cost_rate)
        if self.if_rel_err:
            np.savez('output/'+ timestr + '_' + output_name +'_rel_err.npz', self.rel_err)

# Graphs

In [36]:
%matplotlib notebook
%matplotlib notebook

## Hidden Nodes

In [37]:
def plot_nodes(a, save_name=None, save=True, zlim=True):
    p = a.p
    hmax = a.p[0][0].shape[0]
    col = int(np.sqrt(hmax)) if int(np.sqrt(hmax)) < 10 else 10
    row = np.ceil(hmax / col)
    
    d_x=0.1

    fig = plt.figure(figsize=(col*3, row*3), dpi=100)
    for h in range(1,hmax+1):
        ax = fig.add_subplot(row,col,h, projection='3d')
        x = y = np.arange(0, 1+d_x, d_x)
        X, Y = np.meshgrid(x, y)

        zs = np.array([p[1][h-1]*sig(z(x,y,p))[h-1] for x,y in zip(np.ravel(X), np.ravel(Y))])
        Z = zs.reshape(X.shape)

        ax.set_xlabel('x')
        ax.set_ylabel('y')
        if zlim:
            ax.set_zlim(0,1)

        ax.plot_surface(X, Y, Z, cmap=cm.coolwarm)
        ax.title.set_text('HN: ' + str(h) + ', v=' + "{:.4E}".format(p[1][h-1]))
        time.sleep(0.2)

    if save==True:
        if save_name is None: 
            plt.savefig('t'+str(T)+'_h'+str(hmax)+'_n'+str(a.nx)+'.png')
        else:
            plt.savefig(save_name)
    else:
        plt.show()

## Error vs Iterations

In [38]:
def plot_iteration(arr, min_it=0, max_it=None):
    plt.figure()
    plt.plot(np.arange(len(arr[min_it:max_it])), np.array(arr[min_it:max_it]))
    plt.show()

## Solution Accuracy on the domain

In [39]:
def accuracy(p):
    d_x=0.01
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    x = y = np.arange(0, 1.0+d_x, d_x)
    X, Y = np.meshgrid(x, y)

    zs = np.array([psi_a(x,y)-psi_t(x,y,p) for x,y in zip(np.ravel(X), np.ravel(Y))])
    Z = zs.reshape(X.shape)

    ax.plot_surface(X, Y, Z, cmap=cm.coolwarm_r)
    # ax.ticklabel_format(style='sci', axis='z', scilimits=(0,0), useOffset=True, useMathText=True)


    ax.set_xlabel('x')
    ax.set_ylabel('y')
#     ax.set_zlabel('Accuracy')


    plt.show()

In [40]:
def plot_fun(fun, zlim=False, title=None, return_ax=False, **kwargs):
    d_x=0.01
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    x = y = np.arange(0, 1.0+d_x, d_x)
    X, Y = np.meshgrid(x, y)

    zs = np.array([fun(x,y,**kwargs) for x,y in zip(np.ravel(X), np.ravel(Y))])
    Z = zs.reshape(X.shape)

    ax.plot_surface(X, Y, Z, cmap=cm.coolwarm_r)
    # ax.ticklabel_format(style='sci', axis='z', scilimits=(0,0), useOffset=True, useMathText=True)


    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.title.set_text(title)
    
    if zlim:
        ax.set_zlim(0,1)

    if return_ax:
        return ax
    else:
        plt.show()

# Training

In [41]:
K=10
a = NNTrain(nx=60, bx=300, hidden_nodes=HN_CONST, alpha=1e-4, batch_size=50, if_rel_err=True)

In [42]:
a.train(2)

In [45]:
a.train(10)

         40825958 function calls (40245006 primitive calls) in 70.929 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   478405   20.359    0.000   21.500    0.000 driver.py:291(safe_cuda_api_call)
     5992    5.132    0.001    5.134    0.001 ffi.py:111(__call__)
   397440    2.413    0.000    9.547    0.000 dummyarray.py:159(__init__)
   220800    1.628    0.000   13.969    0.000 devicearray.py:65(__init__)
   147200    1.374    0.000    3.687    0.000 devices.py:144(_get_or_create_context_uncached)
   397440    1.300    0.000    4.514    0.000 dummyarray.py:193(_compute_extent)
   397440    1.200    0.000    1.937    0.000 dummyarray.py:169(_compute_layout)
   309120    1.173    0.000    8.858    0.000 dummyarray.py:149(from_desc)
2124925/1948206    1.144    0.000    1.284    0.000 {built-in method builtins.getattr}
   323840    1.106    0.000    3.291    0.000 driver.py:380(__enter__)
176640/88320    1.095    0.000   45

      133    0.002    0.000    0.012    0.000 packer.py:73(__init__)
     1655    0.002    0.000    0.002    0.000 {method 'encode' of 'str' objects}
      228    0.002    0.000    0.003    0.000 controlflow.py:551(_find_loops)
     1094    0.002    0.000    0.005    0.000 value.py:143(name)
      228    0.002    0.000    0.036    0.000 controlflow.py:114(process)
     1532    0.002    0.000    0.101    0.000 win32.py:104(<genexpr>)
      208    0.002    0.000    0.006    0.000 npydecl.py:24(_handle_inputs)
      814    0.002    0.000    0.006    0.000 builder.py:268(goto_entry_block)
     3289    0.002    0.000    0.002    0.000 instructions.py:639(<listcomp>)
     3964    0.002    0.000    0.003    0.000 models.py:658(get_field_position)
       91    0.002    0.000    0.003    0.000 inspect.py:2092(_signature_from_function)
     1184    0.002    0.000    0.002    0.000 controlflow.py:321(_dfs)
     1242    0.001    0.000    0.005    0.000 values.py:716(_to_list)
      496    0.001   

     1365    0.000    0.000    0.000    0.000 ir.py:1245(terminator)
       31    0.000    0.000    0.061    0.002 codegen.py:148(_get_module_for_linking)
      528    0.000    0.000    0.001    0.000 compiler_machinery.py:25(__exit__)
   270/80    0.000    0.000    0.004    0.000 itanium_mangler.py:155(mangle_type_or_value)
       84    0.000    0.000    0.001    0.000 ntpath.py:471(normpath)
      976    0.000    0.000    0.002    0.000 types.py:121(_to_string)
       19    0.000    0.000    0.288    0.015 codegen.py:199(add_ir_module)
       50    0.000    0.000    0.049    0.001 linker.py:5(link_modules)
        8    0.000    0.000    0.002    0.000 bytecode.py:196(__init__)
       40    0.000    0.000    0.005    0.000 callconv.py:448(_get_return_status)
      167    0.000    0.000    0.001    0.000 module.py:21(_fix_metadata_operands)
      263    0.000    0.000    0.003    0.000 interpreter.py:171(_iter_inst)
      267    0.000    0.000    0.000    0.000 typeinfer.py:824(__setit

      249    0.000    0.000    0.001    0.000 common.py:4(_encode_string)
       50    0.000    0.000    0.000    0.000 module.py:182(functions)
      684    0.000    0.000    0.000    0.000 {method 'copy' of 'dict' objects}
       12    0.000    0.000    0.017    0.001 ir.py:111(strformat)
      194    0.000    0.000    0.002    0.000 itanium_mangler.py:133(<listcomp>)
      691    0.000    0.000    0.000    0.000 context.py:15(_require_nrt)
      800    0.000    0.000    0.000    0.000 templates.py:42(return_type)
      407    0.000    0.000    0.000    0.000 values.py:611(entry_basic_block)
      204    0.000    0.000    0.000    0.000 ufunc_db.py:32(get_ufunc_info)
      446    0.000    0.000    0.000    0.000 typeinfer.py:437(<genexpr>)
      288    0.000    0.000    0.000    0.000 ir.py:190(with_lineno)
     1134    0.000    0.000    0.000    0.000 misc.py:163(key)
       38    0.000    0.000    0.001    0.000 context.py:133(__init__)
       71    0.000    0.000    0.001    0.000

      864    0.000    0.000    0.000    0.000 {method 'keys' of 'dict' objects}
      219    0.000    0.000    0.000    0.000 dataflow.py:778(pop)
      160    0.000    0.000    0.000    0.000 typeinfer.py:1021(propagate_refined_type)
       32    0.000    0.000    0.021    0.001 postproc.py:155(_insert_var_dels)
       33    0.000    0.000    0.000    0.000 interpreter.py:185(_start_new_block)
        8    0.000    0.000    0.000    0.000 typeinfer.py:868(__init__)
       16    0.000    0.000    0.001    0.000 base.py:350(subtarget)
       62    0.000    0.000    0.000    0.000 cpu.py:79(target_data)
       40    0.000    0.000    0.003    0.000 base.py:666(get_returned_value)
      194    0.000    0.000    0.000    0.000 ntpath.py:33(_get_bothseps)
        8    0.000    0.000    0.001    0.000 ssa.py:147(_run_block_analysis)
      484    0.000    0.000    0.000    0.000 instructions.py:133(<listcomp>)
        8    0.000    0.000    0.007    0.001 npyimpl.py:159(_prepare_argument)
   

       79    0.000    0.000    0.000    0.000 base.py:1183(__init__)
       40    0.000    0.000    0.001    0.000 typeinfer.py:928(seed_argument)
      226    0.000    0.000    0.000    0.000 containers.py:305(key)
       65    0.000    0.000    0.000    0.000 {method 'items' of 'mappingproxy' objects}
       36    0.000    0.000    0.000    0.000 _collections_abc.py:879(__iter__)
       32    0.000    0.000    0.000    0.000 compiler_machinery.py:218(_debug_init)
       19    0.000    0.000    0.000    0.000 targets.py:264(add_analysis_passes)
      156    0.000    0.000    0.000    0.000 {method 'with_traceback' of 'BaseException' objects}
       19    0.000    0.000    0.001    0.000 transforms.py:93(populate)
       19    0.000    0.000    0.000    0.000 passmanagers.py:9(create_function_pass_manager)
       16    0.000    0.000    0.001    0.000 templates.py:870(_resolve)
        2    0.000    0.000    0.002    0.001 optional.py:112(optional_to_any)
       16    0.000    0.000   

        2    0.000    0.000    0.003    0.001 arrayobj.py:469(setitem_array)
       13    0.000    0.000    0.000    0.000 types.py:508(<listcomp>)
        8    0.000    0.000    0.001    0.000 genericpath.py:16(exists)
        8    0.000    0.000    0.002    0.000 codegen.py:683(create_library)
        8    0.000    0.000    0.000    0.000 utils.py:238(__init__)
        8    0.000    0.000    0.000    0.000 utils.py:257(__setattr__)
        8    0.000    0.000    0.000    0.000 typed_passes.py:104(legalize_return_type)
        2    0.000    0.000    0.002    0.001 array_exprs.py:386(generate)
       21    0.000    0.000    0.000    0.000 ntpath.py:587(<listcomp>)
      108    0.000    0.000    0.000    0.000 misc.py:222(key)
        8    0.000    0.000    0.000    0.000 ir.py:1400(_reset_analysis_variables)
       28    0.000    0.000    0.000    0.000 bytecode.py:88(get_jump_target)
        8    0.000    0.000    0.000    0.000 inline_closurecall.py:62(__init__)
       66    0.000   

        4    0.000    0.000    0.000    0.000 builtins.py:98(generic)
        8    0.000    0.000    0.000    0.000 ssa.py:171(_make_states)
       20    0.000    0.000    0.000    0.000 multiarray.py:75(empty_like)
       64    0.000    0.000    0.000    0.000 cgutils.py:234(_cast_member_from_value)
        4    0.000    0.000    0.000    0.000 cgutils.py:611(_loop_nest)
       12    0.000    0.000    0.004    0.000 errors.py:296(indicate)
        4    0.000    0.000    0.000    0.000 models.py:912(traverse)
        6    0.000    0.000    0.000    0.000 core.py:89(struct)
        4    0.000    0.000    0.000    0.000 cgutils.py:583(loop_nest)
        2    0.000    0.000    0.001    0.000 base.py:742(make_optional_value)
       13    0.000    0.000    0.000    0.000 values.py:164(<listcomp>)
        6    0.000    0.000    0.000    0.000 ir.py:495(getitem)
        6    0.000    0.000    0.000    0.000 registry.py:49(typing_context)
       13    0.000    0.000    0.000    0.000 dispatche

        8    0.000    0.000    0.000    0.000 controlflow.py:230(nodes)
        2    0.000    0.000    0.000    0.000 controlflow.py:819(op_JUMP_FORWARD)
        2    0.000    0.000    0.000    0.000 ast.py:130(fix_missing_locations)
        2    0.000    0.000    0.000    0.000 cgutils.py:405(terminate)
        6    0.000    0.000    0.000    0.000 numbers.py:1187(bool_as_bool)
        8    0.000    0.000    0.000    0.000 models.py:435(get_data_type)
        2    0.000    0.000    0.000    0.000 ir.py:452(pair_first)
        2    0.000    0.000    0.000    0.000 ir.py:472(iternext)
        4    0.000    0.000    0.000    0.000 containers.py:201(types)
        8    0.000    0.000    0.000    0.000 functions.py:376(<listcomp>)
        2    0.000    0.000    0.000    0.000 core.py:130(undef)
        8    0.000    0.000    0.000    0.000 lowering.py:106(init)
        2    0.000    0.000    0.000    0.000 controlflow.py:792(op_FOR_ITER)
        8    0.000    0.000    0.000    0.000 inline